In [4]:
import sys
import argparse
import os
import pandas as pd
import numpy as np

from collections import Counter

from collections import Counter

pathPrefix = '/Users/friedman/Desktop/mnt'
sys.path.append(pathPrefix + '/ifs/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import mutation_modeling_util  
import clonality_analysis_util

In [11]:
#First load and reformat impact sigs so I can properly work with them
#impactSigs = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/signatures_from_unfiltered_maf.txt')
renameDict = dict([('mean_' + str(i), 'Signature.' + str(i)) for i in range(1,31)] +
[('Tumor_Sample_Barcode', 'Sample Name'), ('Nmut', 'Number of Mutations')])
impactSigs = impactSigs.rename(columns=renameDict)
impactSigs = impactSigs[renameDict.values()]



In [14]:
impactSigs.to_csv(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/impactSigsReformatted.tsv', index=False, sep='\t')
#TODO run the signature attribition script on this


In [35]:
signatureAttribution = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/myAdjustedDataFiles/sigAttribution.tsv')


In [39]:
#assign dicts of each quad nuc
bigD = {}
for col in signatureAttribution.columns.values:
    d = {}
    if col != 'signaturePresentInCase' and col != 'Sample Name':
        d = dict(zip(signatureAttribution['Sample Name'], signatureAttribution[col]))
        bigD[col] = d

In [40]:
allImpactMuts = analysis_utils.load_in_df_with_progress(filePath = pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/all_impact_mutations_annotated_cohort.maf', nLinesFile = 275000)
allImpactMuts['quadNuc'] = allImpactMuts.apply(lambda row: mutationSigUtils.create_reference_four_nuc(row['Ref_Tri'], row['Reference_Allele'], row['Tumor_Seq_Allele2'], row['Variant_Type']), axis=1)

In [50]:
#hotspotMuts = allImpactMuts[allImpactMuts['is-a-hotspot'] == 'Y']
signatureAttribution['APOBEC_present'] = signatureAttribution['signaturesPresentInCase'].apply(lambda x:
                    True if 'APOBEC' in x else False)
apobecCases = signatureAttribution[signatureAttribution['APOBEC_present'] == True]['Sample Name']


In [54]:
def get_attribution(row):
    if row['quadNuc'] not in bigD:
        return None
    littleD = bigD[row['quadNuc']]
    v = littleD[row['Tumor_Sample_Barcode']]
    return v
    
listOfDicts = []
cntr = 0
for case in apobecCases:
    cntr += 1
    caseMuts = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'] == case]
    nmut = caseMuts.shape[0]
    hotspotMuts = caseMuts[caseMuts['is-a-hotspot'] == 'Y']
    if hotspotMuts.shape[0] > 0:
        hotspotMuts['attribute'] = hotspotMuts.apply(lambda row: get_attribution(row), axis=1)
        apobecAttributed = hotspotMuts[hotspotMuts['attribute'].isin(
            ['Signature.APBOEC', 'Signature.APOBEC|Signature.7', 'Signature.7|Signature.APOBEC'])]    
        listOfDicts.append({'nmut': nmut, 'case': case,
                           'nHotspot': hotspotMuts.shape[0], 'nHotspotAPOBEC': apobecAttributed.shape[0]})
  
df = pd.DataFrame(listOfDicts)
        
        

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [56]:
df.to_csv('~/Desktop/WORK/dataForLocalPlotting/hotspotDeterminism.tsv', index=False, sep='\t')